### Download dataset

In [ ]:
import cdsapi

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'format': 'netcdf',
        'variable': [
            '2m_temperature', 'land_sea_mask',
        ],
        'year': '2023',
        'month': '11',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11',
        ],
        'time': [
            '00:00', '01:00', '02:00',
            '03:00', '04:00', '05:00',
            '06:00', '07:00', '08:00',
            '09:00', '10:00', '11:00',
            '12:00', '13:00', '14:00',
            '15:00', '16:00', '17:00',
            '18:00', '19:00', '20:00',
            '21:00', '22:00', '23:00',
        ],
    },
    'download.nc')

### Import libraries

In [ ]:
#install libraries
!pip install xarray
!pip install zarr
!pip install pandas
!pip install pyarrow
!pip install duckdb
!pip install country_bounding_boxes

In [ ]:
#import libraries
import xarray as xr
import zarr
import pandas as pd
import duckdb as db
import time
from country_bounding_boxes import country_subunits_by_iso_code

### Open dataset

In [ ]:
nc_path = "/Users/Sahand/Documents/Research Project/reanalysis-era5-single-levels.nc"

In [ ]:
ds = xr.open_dataset(nc_path)

In [ ]:
#checking dataset variables
ds.variables

### Trasnform to Zarr

In [ ]:
#specify the Zarr file location
zarr_path = "/Users/Sahand/Documents/Research Project/era5-single-levels.zarr"

In [ ]:
ds.to_zarr(zarr_path)

### Open Zarr file

In [ ]:
xarr = xr.open_zarr(zarr_path)

### Transform to Parquet

In [ ]:
df = xarr.to_dataframe()

In [ ]:
#specify the Parquet file location
parquet_path = '/Users/amal2174/Downloads/code rp/era5-single-levels.parquet'

In [ ]:
df.to_parquet(parquet_path)

In [ ]:
ds.close()

### Dataset info

In [ ]:
#read parquet file
df = pd.read_parquet(parquet_path)

In [ ]:
#observing the first five rows
df.head(n=5)

In [ ]:
#observing the last five rows
df.tail(n=5)

In [ ]:
parquet_file = pq.ParquetFile(parquet_path)
schema = parquet_file.schema
#checking the dataset schema
schema

In [ ]:
#checking the dataset columns
df.columns

### Resolve columns problem

In [ ]:
df.reset_index(inplace=True, names=['latitude', 'longitude', 'time', 'lsm', 't2m'])

In [ ]:
#checking the dataset columns
df.columns

In [ ]:
#checking the first five rows
df.head(n=5)

In [ ]:
#checking the last five rows
df.tail(n=5)

In [ ]:
#checking the datatypes and data in closer look
#pq.read_table(parquet_path)

### duckdb connect

In [ ]:
db_path = '/Users/amal2174/Downloads/code rp/era5_v4.db'

In [ ]:
con = db.connect(
    database = db_path,
    read_only = False,
    config = {
        "memory_limit": '16GB'
    }
)

In [ ]:
#enables the progress bar, printing progress to the terminal for long queries
con.execute("SET enable_progress_bar = true;")

### Create Dataframe

In [ ]:
#creating dataframe for era5
era5_df = pd.DataFrame({
    'id': range(1, len(df) + 1),
    'lat': df['latitude'].explode(),
    'lon': df['longitude'].explode(),
    'time': df['time'].explode(),
    'lsm': df['lsm'].explode(),
    't2m': df['t2m'].explode()
})

In [ ]:
#save the era5 datafram as a Parquet file
parquet_path = '/Users/Sahand/Documents/Research Project/era5.parquet'
era5_df.to_parquet(parquet_path, compression='zstd')

In [ ]:
#read parquet file
era5_df = pd.read_parquet(parquet_path)

### Create tables

#### New design 3NF (not in report)

In [ ]:
table_name = 'coord'
view_name = 'coordV'
coord_df = df[['latitude', 'longitude']].drop_duplicates()
con.execute("CREATE SEQUENCE coord_seq START 1")
coord_df['cid'] = coord_df.apply(lambda row: con.execute("SELECT NEXTVAL('coord_seq')").fetchone()[0], axis=1)

con.register(view_name, coord_df)

#create table
con.execute(f'CREATE TABLE {table_name} (latitude FLOAT, longitude FLOAT, cid INT PRIMARY KEY)')

#start timer for creating table in database
start_time = time.time()
con.execute(f'INSERT INTO {table_name} SELECT * FROM {view_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

In [ ]:
table_name = 'lsmask'
view_name = 'lsmaskV'
lsm_df = df[['lsm']].drop_duplicates()
con.execute("CREATE SEQUENCE lsm_seq START 1")
lsm_df['lid'] = lsm_df.apply(lambda row: con.execute("SELECT NEXTVAL('lsm_seq')").fetchone()[0], axis=1)

con.register(view_name, lsm_df)

#create table
con.execute(f'CREATE TABLE {table_name} (lsm FLOAT, lid INT PRIMARY KEY)')

#start timer for creating table in database
start_time = time.time()
con.execute(f'INSERT INTO {table_name} SELECT * FROM {view_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

In [ ]:
table_name = 'temp_data'
view_name = 'tempV'

# Define the DataFrame for the temperature data
temp_df = df[['latitude', 'longitude', 'time', 'lsm', 't2m']].copy()

# Add cid to temp_df by joining with coord_df
temp_df = temp_df.merge(coord_df, on=['latitude', 'longitude'])

# Add lid to temp_df by joining with lsm_df
temp_df = temp_df.merge(lsm_df, on=['lsm'])

# Drop latitude, longitude, and lsm columns as they're already referenced in other tables
temp_df.drop(['latitude', 'longitude', 'lsm'], axis=1, inplace=True)

# Register the temp_df as a DuckDB table
con.register(view_name, temp_df)

# Create the third table with foreign key references to coord_id and lsm_id
con.execute(f'CREATE TABLE {table_name} (time TIMESTAMP, t2m FLOAT, cid INT, lid INT, FOREIGN KEY(cid) REFERENCES coord(cid), FOREIGN KEY(lid) REFERENCES lsmask(lid))')

#start timer for creating table in database
start_time = time.time()
con.execute(f'INSERT INTO {table_name} SELECT * FROM {view_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

#### Design 1: without normalization and categorization

In [ ]:
#create table based on original dataframe
table_name = 'era5'
con.register(table_name, era5_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

### Design 2: third normal form

In [ ]:
#creating coordination dataframe
coord_df = era5_df[['id', 'lat', 'lon']].copy()

In [ ]:
#creating table for coordination
table_name = 'coord'
con.register(table_name, coord_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

In [ ]:
#creating land-sea mask dataframe
lsm_df = era5_df[['id', 'time', 'lsm']].copy()

In [ ]:
#creating table for land-sea mask
table_name = 'lsm'
con.register(table_name, lsm_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")
# Add FOREIGN KEY constraint
#con.execute(f'ALTER TABLE {table_name} ADD CONSTRAINT fk_coord_id FOREIGN KEY (coord_id) REFERENCES coord(id)')

In [ ]:
#creating 2 meter temperature dataframe
t2m_df = era5_df[['id', 'time', 't2m']].copy()

In [ ]:
#creating table for 2 meter temperature
table_name = 't2m'
con.register(table_name, t2m_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")
# Add FOREIGN KEY constraint
#con.execute(f'ALTER TABLE {table_name} ADD CONSTRAINT fk_coord_id FOREIGN KEY (coord_id) REFERENCES coord(id)')

### Design 3: categorization based on regions

In [ ]:
#creating categorical dataframe
cat_df = era5_df[['id', 'time', 'lsm', 't2m']].copy()

##### North Europe

In [ ]:
# Define country codes for the specified countries
north_Europe = ["DK", "EE", "FI", "IS", "IE", "LV", "LT", "NO", "SE", "GB"]

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in north_Europe:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, north_Europe):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'north_Europe'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### South Europe

In [ ]:
# Define country codes for the specified countries
south_Europe = ['AL', 'AD', 'BA', 'HR', 'CY', 'GR', 'IT', 'MT', 'ME', 'MK', 'PT', 'SM', 'RS', 'SI', 'ES', 'VA']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in south_Europe:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Europe):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_Europe'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### West Europe

In [ ]:
# Define country codes for the specified countries
west_Europe = ['AT', 'BE', 'FR', 'DE', 'LU', 'NL', 'CH']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in south_Europe:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, west_Europe):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'west_Europe'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### East Europe

In [ ]:
# Define country codes for the specified countries
east_Europe = ['BG', 'BY', 'CZ', 'HU', 'MD', 'PL', 'RO', 'RU', 'SK', 'UA']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in east_Europe:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, east_Europe):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'west_Europe'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### North America

In [ ]:
# Define country codes for the specified countries
north_America = ['US', 'CA', 'MX', 'GT', 'HN', 'CR', 'PA', 'BZ', 'SV', 'NI']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in north_America:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, north_America):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'north_America'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### South America

In [ ]:
# Define country codes for the specified countries
south_America = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'GY', 'PY', 'PE', 'SR', 'UY', 'VE']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in south_America:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_America):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_America'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### North Asia

In [ ]:
# Define country codes for the specified countries
north_Asia = ['RU', 'KZ', 'MN']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in north_Asia:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, north_Asia):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'north_Asia'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### South Asia

In [ ]:
# Define country codes for the specified countries
south_Asia = ['IN', 'PK', 'BD', 'NP', 'LK', 'BT']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in south_Asia:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Asia):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_Asia'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### West Asia

In [ ]:
# Define country codes for the specified countries
west_Asia = ['SA', 'IQ', 'IR', 'KW', 'QA', 'OM', 'AE', 'SY', 'LB', 'YE', 'JO', 'PS']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in west_Asia:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Asia):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_Asia'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### East Asia

In [ ]:
# Define country codes for the specified countries
east_Asia = ['CN', 'JP', 'KR', 'TW', 'MN', 'HK', 'MO']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in east_Asia:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Asia):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'east_Asia'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### North Africa

In [ ]:
# Define country codes for the specified countries
north_Africa = ['DZ', 'EG', 'LY', 'MA', 'TN']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in north_Africa:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, north_Africa):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'north_Africa'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### South Africa

In [ ]:
# Define country codes for the specified countries
south_Africa = ['ZA', 'BW', 'LS', 'NA', 'SZ']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in south_Africa:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Africa):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_Africa'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### West Africa

In [ ]:
# Define country codes for the specified countries
west_Africa = ['NG', 'GH', 'CI', 'SN', 'BF', 'ML', 'GM', 'GN', 'GW', 'LR', 'SL', 'MR', 'CV', 'SH']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in west_Africa:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, south_Africa):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'south_Africa'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### East Africa

In [ ]:
# Define country codes for the specified countries
east_Africa = ['ET', 'KE', 'UG', 'TZ', 'RW', 'BI', 'DJ', 'ER', 'SO', 'YT', 'KM', 'RE', 'MU', 'SC', 'SS', 'SD']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in east_Africa:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, east_Africa):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'east_Africa'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

##### Oceania

In [ ]:
# Define country codes for the specified countries
oceania = ['AS', 'AU', 'CK', 'FJ', 'PF', 'GU', 'KI', 'MH', 'FM', 'NR', 'NC', 'NZ', 'NU', 'NF', 'MP', 'PW', 'PG', 'PN', 'WS', 'SB', 'TK', 'TO', 'TV', 'VU', 'WF']

# Initialize an empty list to store bounding boxes
bounding_boxes = []

# Iterate over the country codes and convert the set iterator to a list
for code in oceania:
    # Convert the set iterator to a list and append the first element (bounding box)
    bounding_boxes.append(list(country_subunits_by_iso_code(code))[0])

# Initialize an empty DataFrame to store filtered coordinates
filtered_coords = pd.DataFrame()

# Function to filter coordinates based on bounding box
def filter_coords(bbox, coord_df):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Filtering coordinates based on bounding box and appending to filtered_coords
    return coord_df[(coord_df['lat'] >= lat_min) & 
                    (coord_df['lat'] <= lat_max) &
                    (coord_df['lon'] >= lon_min) &
                    (coord_df['lon'] <= lon_max)]

# Iterate over the bounding boxes
for bbox in bounding_boxes:
    filtered_coords = pd.concat([filtered_coords, filter_coords(bbox, coord_df)], ignore_index=True)

# Add a new column 'ccode' filled with corresponding country codes
filtered_coords['ccode'] = ''

# Iterate over the bounding boxes and assign country codes
for bbox, code in zip(bounding_boxes, oceania):
    # Accessing latitude and longitude from bbox object
    lat_min, lon_min, lat_max, lon_max = bbox.bbox
    
    # Assigning country code to corresponding rows
    filtered_coords.loc[(filtered_coords['lat'] >= lat_min) & 
                        (filtered_coords['lat'] <= lat_max) &
                        (filtered_coords['lon'] >= lon_min) &
                        (filtered_coords['lon'] <= lon_max), 'ccode'] = code

# Joining filtered_coords with cat_df to get the corresponding data
result_df = filtered_coords.merge(cat_df, on=['id'])

# Define the table name
table_name = 'oceania'

# Selecting only the desired columns
result_df = result_df[['id', 'time', 'lsm', 't2m', 'ccode']]

#register the DataFrame as a DuckDB table
con.register(table_name, result_df)

#start timer for creating table in database
start_time = time.time()
con.execute(f'CREATE TABLE {table_name} AS SELECT * FROM {table_name}')
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")

print(f"Table '{table_name}' created successfully.")

In [ ]:
storage_info = con.execute("PRAGMA database_size;").fetchone()
print(f"Database size: '{storage_info[1]}'")

### Query

In [ ]:
con.execute("SHOW TABLES;")
print(con.fetchall())

#### Query 1

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT lat, lon, (t2m - 273.15) AS temp_celsius \
                        FROM era5 \
                        ORDER BY temp_celsius \
                        LIMIT 10;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT coord.lat, coord.lon, (t2m.t2m - 273.15) AS temp_celsius \
                        FROM coord \
                        JOIN t2m ON coord.id == t2m.id \
                        ORDER BY temp_celsius \
                        LIMIT 10;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT id, time, lsm, t2m, ccode \
                        FROM ( \
                        SELECT id, time, lsm, t2m, ccode \
                        FROM north_Europe \
                        UNION ALL \
                        SELECT id, time, lsm, t2m, ccode \
                        FROM south_Europe \
                        ) AS combined_regions \
                        ORDER BY t2m ASC \
                        LIMIT 10;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 2

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT coord.lat, coord.lon, (t2m.t2m - 273.15) AS temp_celsius \
                        FROM era5 \
                        GROUP BY year \
                        ORDER BY year;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT YEAR(t2m.time) AS year, AVG(t2m.t2m - 273.15) AS avg_temp_celsius \
                        FROM temp_data \
                        JOIN coord ON t2m.id == coord.id \
                        GROUP BY year \
                        ORDER BY year;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(YEAR FROM time) AS year, \
    AVG(t2m - 273.15) AS avg_temp_celsius, \
    ccode \
FROM \
    ( \
    SELECT id, time, t2m, ccode \
    FROM north_Europe \
    UNION ALL \
    SELECT id, time, t2m, ccode \
    FROM south_Europe \
    ) AS combined_regions \
JOIN \
    t2m ON combined_regions.id = t2m.id \
GROUP BY \
    year, ccode \
ORDER BY \
    year, ccode;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 3

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT lsm, AVG(t2m - 273.15) AS avg_temp_celsius \
FROM era5 \
WHERE lsm > 0.5 \
GROUP BY lsm \
ORDER BY avg_temp_celsius DESC;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT lsm, AVG(t2m - 273.15) AS avg_temp_celsius \
FROM temp_data \
JOIN lsm ON temp_data.lsm_id == lsm.lsm_id \
WHERE lsm > 0.5 \
GROUP BY lsm \
ORDER BY avg_temp_celsius DESC;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lsm, \
    AVG(t2m - 273.15) AS avg_temp_celsius, \
    ccode \
FROM \
    ( \
    SELECT id, time, lsm, t2m, ccode \
    FROM north_Europe \
    WHERE lsm > 0.5 \
    UNION ALL \
    SELECT id, time, lsm, t2m, ccode \
    FROM south_Europe \
    WHERE lsm > 0.5 \
    -- Add more region-based tables here if needed \
    ) AS combined_regions \
GROUP BY \
    lsm, ccode \
ORDER BY \
    lsm, ccode;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 4

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lat, \
    lon, \
    time, \
    (t2m - LAG(t2m) OVER (PARTITION BY lat, lon ORDER BY times)) AS temp_difference_celsius \
FROM \
    era5;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    latitude, \
    longitude, \
    t2m.time, \
    (t2m.t2m - LAG(t2m.t2m) OVER (PARTITION BY coord.lat, coord.lon ORDER BY t2m.time)) AS temp_difference_celsius \
FROM \
    t2m \
JOIN \
    coord ON t2m.id = coord.id;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lsm, \
    t2m.time, \
    (t2m.t2m - LAG(t2m.t2m) OVER (PARTITION BY lsm, ccode ORDER BY t2m.time)) AS temp_difference_celsius, \
    ccode \
FROM \
    ( \
    SELECT id, time, lsm, t2m, ccode \
    FROM north_Europe \
    UNION ALL \
    SELECT id, time, lsm, t2m, ccode \
    FROM south_Europe \
    -- Add more region-based tables here if needed \
    ) AS combined_regions \
ORDER BY \
    lsm, ccode, t2m.time;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 5

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(MONTH FROM time) AS month, \
    AVG(t2m - 273.15) AS avg_temp_celsius \
FROM \
    era5 \
GROUP BY \
    month \
ORDER BY \
    month;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT EXTRACT(MONTH FROM t2m.time) AS month, \
       AVG(t2m.t2m - 273.15) AS avg_temp_celsius \
FROM t2m \
JOIN coord ON t2m.id == coord.id \
GROUP BY month \
ORDER BY month;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(MONTH FROM time) AS month, \
    AVG(t2m - 273.15) AS avg_temp_celsius, \
    ccode \
FROM \
    ( \
    SELECT id, time, t2m, ccode \
    FROM north_Europe \
    UNION ALL \
    SELECT id, time, t2m, ccode \
    FROM south_Europe \
    -- Add more region-based tables here if needed \
    ) AS combined_regions \
GROUP BY \
    month, ccode \
ORDER BY \
    month, ccode;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 6

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lat, \
    lon, \
    MAX(t2m - 273.15) - MIN(t2m - 273.15) AS temp_fluc_celsius \
FROM \
    era5 \
GROUP BY \
    lat, lon \
ORDER BY \
    temp_fluc_celsius DESC \
LIMIT 5;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \ 
    coord.lat, \
    coord.lon, \
    (MAX(t2m.t2m) - MIN(t2m.t2m)) - 273.15 AS temp_fluc_celsius \
FROM \
    era5 \
JOIN \
    coord ON era5.id = coord.id \
JOIN \
    t2m ON era5.id = t2m.id \
GROUP BY \
    coord.lat, coord.lon \
ORDER BY \
    temp_fluc_celsius DESC \
LIMIT 5;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    ccode, \
    AVG(MAX(t2m) - MIN(t2m)) AS temp_fluctuation \
FROM \
    ( \
    SELECT ccode, time, MAX(t2m) AS max_temp, MIN(t2m) AS min_temp \
    FROM north_Europe \
    GROUP BY ccode, time \
    UNION ALL \
    SELECT ccode, time, MAX(t2m) AS max_temp, MIN(t2m) AS min_temp \
    FROM south_Europe \
    GROUP BY ccode, time \
    -- Add more region-based tables here if needed \
    ) AS combined_regions \
GROUP BY \
    ccode \
ORDER BY \
    temp_fluctuation DESC \
LIMIT 5;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 7

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(HOUR FROM timestamp) AS hour, \
    AVG(t2m - 273.15) AS avg_temp_celsius \
FROM \
    era5 \
GROUP BY \
    hour \
ORDER BY \
    hour;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(HOUR FROM t2m.time) AS hour, \
    AVG(t2m.t2m - 273.15) AS avg_temp_celsius \
FROM \
    t2m \
JOIN \
    coord ON t2m.id = coord.id \
GROUP BY \
    hour \
ORDER BY \
    hour;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    EXTRACT(HOUR FROM time) AS hour, \
    AVG(t2m) AS avg_temp \
FROM \
    ( \
    SELECT time, t2m, ccode \
    FROM north_Europe \
    UNION ALL \
    SELECT time, t2m, ccode \
    FROM south_Europe \
    -- Add more region-based tables here if needed \
    ) AS combined_regions \
GROUP BY \
    hour \
ORDER BY \
    hour;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

#### Query 8

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lat, \
    lon \
FROM ( \
    SELECT \ 
        lat, \ 
        lon, \
        time, \
        t2m - LAG(t2m) OVER (PARTITION BY lat, lon ORDER BY time) AS temp_difference_celsius, \
        ROW_NUMBER() OVER (PARTITION BY lat, lon ORDER BY time) AS rn \
    FROM \
        era5 \
) AS temp_changes \
WHERE \
    temp_difference_celsius > 0 \
GROUP BY \
    lat, lon, rn \
HAVING \
    COUNT(*) >= 3;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)   

In [ ]:
import time
# start timer for creating table in database
start_time = time.time()
result = con.execute("SELECT \
    lat, lon \
FROM ( \
    SELECT \
        coord.lat, \ 
        coord.lon, \
        t2m.time, \
        t2m.t2m - LAG(t2m.t2m) OVER (PARTITION BY coord.lat, coord.lon ORDER BY t2m.time) AS temp_difference_celsius, \
        ROW_NUMBER() OVER (PARTITION BY coord.lat, coord.lon ORDER BY t2m.time) AS rn \
    FROM \
        t2m \
    JOIN \
        coord ON t2m.id = coord.id \
) AS temp_changes \
WHERE \
    temp_difference_celsius > 0 \
GROUP BY \
    lat, lon, rn \
HAVING \
    COUNT(*) >= 3;")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

In [ ]:
#start timer for creating table in database
start_time = time.time()
result = con.execute("WITH TempChanges AS ( \
    SELECT \
        ccode, \
        time, \
        t2m, \
        LAG(t2m) OVER (PARTITION BY ccode ORDER BY time) AS prev_temp \
    FROM \
        (
        SELECT time, t2m, ccode \
        FROM north_Europe \
        UNION ALL \
        SELECT time, t2m, ccode \
        FROM south_Europe \
        -- Add more region-based tables here if needed \
        ) AS combined_regions \
)\
SELECT \
    ccode \
FROM \
    TempChanges \
WHERE \
    prev_temp IS NOT NULL AND t2m > prev_temp \
GROUP BY \
    ccode \
HAVING \
    COUNT(*) = (SELECT COUNT(*) - 1 FROM TempChanges WHERE ccode = TempChanges.ccode);")
execution_time = time.time() - start_time
print(f"Execution time: {execution_time} seconds")
re_df = result.fetchdf()
print(re_df)

### Close database

In [ ]:
con.close()